<a href="https://colab.research.google.com/github/being-harisz/Skill-Development-Course/blob/main/CNN-LSTM-RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install gradio
!pip install gradio tensorflow numpy



In [6]:
import numpy as np
import gradio as gr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# Prediction function
def predict_digit(img):
    img = img.resize((28, 28)).convert("L")
    img = np.array(img)
    img = img.reshape(1, 28, 28, 1) / 255.0
    prediction = model.predict(img)[0]
    return {str(i): float(prediction[i]) for i in range(10)}

# Gradio interface
gr.Interface(
    fn=predict_digit,
    inputs=gr.Image(image_mode="L", source="canvas", tool="editor"),
    outputs=gr.Label(num_top_classes=3),
    title="Digit Recognizer with CNN"
).launch()


Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 21ms/step - accuracy: 0.9069 - loss: 0.3016 - val_accuracy: 0.9782 - val_loss: 0.0687
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 38s 20ms/step - accuracy: 0.9838 - loss: 0.0524 - val_accuracy: 0.9785 - val_loss: 0.0613


TypeError: Image.__init__() got an unexpected keyword argument 'source'

In [ ]:
import numpy as np
import gradio as gr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam

# Step 1: Prepare sequence data (using a simple sine wave example)
def generate_sequence(n_steps=100):
    x = np.linspace(0, 10, n_steps)
    y = np.sin(x)
    return x, y

# Step 2: Prepare training data for RNN
n_steps = 100
x, y = generate_sequence(n_steps)
x_train = np.array([x[:-1]])
y_train = np.array([y[1:]])

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

# Step 3: Build the RNN model
model = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(x_train.shape[1], 1)),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Step 4: Train the model
model.fit(x_train, y_train, epochs=50, batch_size=1, verbose=0)

# Step 5: Prediction function
def predict_next_value(input_sequence):
    input_sequence = np.array(input_sequence).reshape((1, len(input_sequence), 1))
    predicted_value = model.predict(input_sequence)
    return f"Predicted Next Value: {predicted_value[0][0]:.4f}"

# Step 6: Gradio interface
gr.Interface(
    fn=predict_next_value,
    inputs=gr.Textbox(label="Enter sequence (comma separated)", placeholder="e.g. 0.0, 0.5, 1.0, ..."),
    outputs=gr.Textbox(label="Predicted Next Value"),
    title="RNN Sequence Predictor",
    description="Input a sequence of numbers (comma separated) and predict the next value using an RNN model."
).launch()


In [ ]:
import numpy as np
import gradio as gr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Step 1: Prepare sequence data (using a sine wave example)
def generate_sequence(n_steps=100):
    x = np.linspace(0, 10, n_steps)
    y = np.sin(x)
    return x, y

# Step 2: Prepare training data for LSTM
n_steps = 100
x, y = generate_sequence(n_steps)

# Reshape data to fit LSTM input format
x_train = np.array([x[:-1]])
y_train = np.array([y[1:]])

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

# Step 3: Build the LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(x_train.shape[1], 1)),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Step 4: Train the model
model.fit(x_train, y_train, epochs=50, batch_size=1, verbose=0)

# Step 5: Prediction function
def predict_next_value(input_sequence):
    input_sequence = np.array(input_sequence).reshape((1, len(input_sequence), 1))
    predicted_value = model.predict(input_sequence)
    return f"Predicted Next Value: {predicted_value[0][0]:.4f}"

# Step 6: Gradio interface
gr.Interface(
    fn=predict_next_value,
    inputs=gr.Textbox(label="Enter sequence (comma separated)", placeholder="e.g. 0.0, 0.5, 1.0, ..."),
    outputs=gr.Textbox(label="Predicted Next Value"),
    title="LSTM Sequence Predictor",
    description="Input a sequence of numbers (comma separated) and predict the next value using an LSTM model."
).launch()


In [ ]:
import can
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# Step 1: Set up CAN interface
bus = can.interface.Bus(bustype='socketcan', channel='can0', bitrate=500000)

# Step 2: Collect data (e.g., engine speed and temperature)
def collect_can_data():
    messages = []
    for _ in range(100):  # Collect 100 messages
        msg = bus.recv()  # Read CAN message
        if msg.arbitration_id == 0x123:  # Filter for specific CAN ID
            data = list(msg.data)  # Get the data from the message
            messages.append(data)
    return np.array(messages)

# Step 3: Preprocess the data
def preprocess_data(raw_data):
    # Example preprocessing: scale the data
    scaler = StandardScaler()
    return scaler.fit_transform(raw_data)

# Step 4: Define a simple neural network model
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  # Regression (e.g., speed)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 5: Train the model on collected data
def train_model():
    raw_data = collect_can_data()
    data = preprocess_data(raw_data)
    labels = data[:, 0]  # Assume first column is the target (e.g., speed)
    data = data[:, 1:]   # Use the rest of the data as features

    model = create_model(input_shape=(data.shape[1],))
    model.fit(data, labels, epochs=10, batch_size=32)

    return model

# Step 6: Prediction with the trained model
def predict(model, input_data):
    prediction = model.predict(input_data)
    return prediction

# Example of training the model and predicting
model = train_model()
test_data = np.array([[0.5, 1.2, 3.4]])  # Example test data
predicted_value = predict(model, test_data)
print(f"Predicted Value: {predicted_value}")
